# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
#from matplotlib_venn import venn2
import pandas_datareader # install with `pip install pandas-datareader`
from dstapi import DstApi # install with `pip install git+https://github.com/alemartinello/dstapi`

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

**We import data from Statistics Denmark regarding production divided between industries and data regarding the effective exchange rate of the DKK**

In [10]:
GDP = DstApi('NKBP10')
exrate = DstApi('DNVALQ') 

# a. Tables of the variables
tabsum_GDP = GDP.tablesummary(language='en')
display(tabsum_GDP)
tabsum_exrate = exrate.tablesummary(language='en')
display(tabsum_exrate)


Table NKBP10: 1-2.1.1 Production
and  generation of income (10a3-grouping) by transaction, industry, price unit, seasonal adjustment and time
Last update: 2023-03-31T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,TRANSAKT,7,P1K,P.1 Output,B2A3GD,B.2g+B.3g Gross operating surplus and mixed in...,False
1,BRANCHE,15,V,Total,VR_S,"R_S Arts, entertainment and other services",False
2,PRISENHED,2,V,Current prices,LKV,"2010-prices, chained values",False
3,SÆSON,2,N,Non-seasonally adjusted,Y,Seasonally adjusted,False
4,Tid,132,1990K1,1990Q1,2022K4,2022Q4,True


Table DNVALQ: Quarterly exchange rates by currency, type, methodology and time
Last update: 2023-03-30T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,VALUTA,2,DKK,Nominal effective Krone rate (Jan. 1970-),X00,All currencies - excl. Danish kroner,False
1,KURTYP,4,INX,"Index (only nominal effective krone rate), ind...",RET,Real effective krone rate based on hourly earn...,False
2,OPGOER,3,A,Quarterly average,Y,Annual growth rate,False
3,Tid,212,1970K1,1970Q1,2022K4,2022Q4,True


## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

In [11]:
# The available values for a each variable in GDP dataset: 
for variable in tabsum_GDP['variable name']:
    print(variable+':')
    display(GDP.variable_levels(variable, language='en'))

TRANSAKT:


,id,text
0,P1K,P.1 Output
1,P2D,P.2 Intermediate consumption
2,B1GD,B.1g Gross value added
3,D29X39D,D.29-D.39 Other taxes less subsidies on produc...
4,B1GFD,B.1GF Gross domestic product at factor cost
5,D1D,D.1 Compensation of employees
6,B2A3GD,B.2g+B.3g Gross operating surplus and mixed in...


BRANCHE:


,id,text
0,V,Total
1,VMEMO,Of which: General government
2,VA,"A Agriculture, forestry and fishing"
3,VB,B Mining and quarrying
4,VC,C Manufacturing
5,VD_E,D_E Utility services
6,VF,F Construction
7,VG_I,G_I Trade and transport etc.
8,VJ,J Information and communication
9,VK,K Financial and insurance


PRISENHED:


,id,text
0,V,Current prices
1,LKV,"2010-prices, chained values"


SÆSON:


,id,text
0,N,Non-seasonally adjusted
1,Y,Seasonally adjusted


Tid:


,id,text
0,1990K1,1990Q1
1,1990K2,1990Q2
2,1990K3,1990Q3
3,1990K4,1990Q4
4,1991K1,1991Q1
...,...,...
127,2021K4,2021Q4
128,2022K1,2022Q1
129,2022K2,2022Q2
130,2022K3,2022Q3


In [12]:
# The available values for each variable in exchange rate dataset: 
for variable in tabsum_exrate['variable name']:
    print(variable+':')
    display(exrate.variable_levels(variable, language='en'))

VALUTA:


,id,text
0,DKK,Nominal effective Krone rate (Jan. 1970-)
1,X00,All currencies - excl. Danish kroner


KURTYP:


,id,text
0,INX,"Index (only nominal effective krone rate), ind..."
1,LOI,"Hourly earnings in manufacturing in Denmark, s..."
2,LOU,Weighted hourly earnings in manufacturing abro...
3,RET,Real effective krone rate based on hourly earn...


OPGOER:


,id,text
0,A,Quarterly average
1,B,Calculated
2,Y,Annual growth rate


Tid:


,id,text
0,1970K1,1970Q1
1,1970K2,1970Q2
2,1970K3,1970Q3
3,1970K4,1970Q4
4,1971K1,1971Q1
...,...,...
207,2021K4,2021Q4
208,2022K1,2022Q1
209,2022K2,2022Q2
210,2022K3,2022Q3


**Defining parameters, joining and filtering data**

In [13]:
# a. define parameters
params_GDP = GDP._define_base_params(language='en')
params_exrate = exrate._define_base_params(language='en')

# b. load api
GDP_api = GDP.get_data(params=params_GDP)
exrate_api = exrate.get_data(params=params_exrate)

In [26]:
# a. left join data by TID
api_inner = pd.merge(GDP_api, exrate_api, on='TID', how='left')
api_inner.rename(columns = {'INDHOLD_x':'INDHOLD_GDP', 'INDHOLD_y':'INDHOLD_exrate'}, inplace=True)

# b. filter data
I = api_inner.TRANSAKT.str.contains('P.1')
I |= api_inner.TRANSAKT.str.contains('D.29-D.39')
I |= api_inner.TRANSAKT.str.contains('D.1')
I &= api_inner.PRISENHED.str.contains('Current prices')
I &= api_inner.SÆSON.str.contains('Non-seasonally')
I &= api_inner.VALUTA.str.contains('Nominal effective')
I &= api_inner.KURTYP.str.contains('Index')
I &= api_inner.OPGOER.str.contains('Quarterly')
api_inner.loc[I, :]
api_inner = api_inner.loc[I == True]

# c. new indexing
api_inner.reset_index(inplace = True, drop = True)

TRANSAKT           object
BRANCHE            object
PRISENHED          object
SÆSON              object
TID                object
INDHOLD_GDP       float64
VALUTA             object
KURTYP             object
OPGOER             object
INDHOLD_exrate    float64
dtype: object

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

In [29]:
api_inner.astype({'INDHOLD_GDP':'float','INDHOLD_exrate':'float'}).describe()


,INDHOLD_GDP,INDHOLD_exrate
count,5.940000e+03,5940.000000
mean,4.450545e+04,101.246656
std,1.174763e+05,3.080568
min,-1.095000e+04,93.530100
25%,2.680000e+02,99.452925
50%,8.443000e+03,101.712800
75%,3.779875e+04,103.231075
max,1.380925e+06,108.345800


MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.